In [1]:
import os
import time

In [2]:
os.environ['KERAS_BAXKEND']='tensorflow'

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import keras
import keras_nlp
import tensorflow as tf

In [4]:
keras.mixed_precision.set_global_policy("mixed_float16")
print(tf.__version__)

2.16.1


In [5]:
root = '/Users/pals/MICS/MICS_207/Project'
wikitext2_root = root + '/datasets/kaggle-wikitext/wikitext-2/'
train_file = wikitext2_root + 'wiki.train.tokens'
test_file  = wikitext2_root + 'wiki.test.tokens'
valid_file = wikitext2_root + 'wiki.valid.tokens'
train_data =[]
test_data  = []

In [6]:
BATCH_SIZE = 128
NUM_EPOCHS = 3
SEQUENCE_LENGTH = 128

In [7]:
with open(train_file) as fd:
    for line in fd:
        train_data.append(line)

In [8]:
with open(test_file) as fd:
    for line in fd:
        test_data.append(line)

In [9]:
X_train = tf.data.Dataset.from_tensor_slices((train_data)).batch(BATCH_SIZE).cache().prefetch(tf.data.AUTOTUNE)
X_test  = tf.data.Dataset.from_tensor_slices((test_data)).batch(BATCH_SIZE).cache().prefetch(tf.data.AUTOTUNE)

In [10]:
print(X_train.cardinality())

tf.Tensor(287, shape=(), dtype=int64)


In [11]:
preprocessor = keras_nlp.models.GPT2CausalLMPreprocessor.from_preset(
    "gpt2_base_en",
    sequence_length=SEQUENCE_LENGTH,
)
gpt2_lm = keras_nlp.models.GPT2CausalLM.from_preset(
    "gpt2_base_en", preprocessor=preprocessor
)

In [12]:
gpt2_lm.summary()

Preprocessor: "gpt2_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gpt2_tokenizer (GPT2Tokenizer)                     │                                              50,257 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gpt2_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gpt2_backbone (GPT2Backbone)  │ (None, None, 768)         │     124,439,808 │ padding_mask[0][0],        │
│                               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 50257)       │      38,597,376 │ gpt2_backbone[0][0]        │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 124,439,808 (474.70 MB)

 Trainable params: 124,439,808 (474.70 MB)

 Non-trainable params: 0 (0.00 B)

In [13]:
learning_rate = keras.optimizers.schedules.PolynomialDecay(
    5e-5,
    decay_steps=X_train.cardinality() * NUM_EPOCHS,
    end_learning_rate=0.0,
)
loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True)
gpt2_lm.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss=loss,
    weighted_metrics=["accuracy"],
)

In [14]:
st = time.time()
gpt2_lm.fit(X_train, epochs=NUM_EPOCHS)
en = time.time()

Epoch 1/3
287/287 ━━━━━━━━━━━━━━━━━━━━ 15592s 54s/step - accuracy: 0.3790 - loss: 1.3349
Epoch 2/3
287/287 ━━━━━━━━━━━━━━━━━━━━ 15524s 54s/step - accuracy: 0.4212 - loss: 1.1855
Epoch 3/3
287/287 ━━━━━━━━━━━━━━━━━━━━ 15230s 53s/step - accuracy: 0.4304 - loss: 1.1537


In [15]:
print(f'Training time {(en-st)/3600} hours')

Training time 12.874009280867046 hours


In [16]:
result = gpt2_lm.generate("My trip to Yosemite was", max_length=200)

I0000 00:00:1719594341.773260 17920434 service.cc:145] XLA service 0x6000058bc300 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1719594341.773723 17920434 service.cc:153]   StreamExecutor device (0): Host, Default Version
I0000 00:00:1719594341.865253 17920434 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


In [17]:
print(result)

My trip to Yosemite was an adventure in itself. I had never seen a place so beautiful and so full of life . The view from the top of the mountain and the beauty of the mountain made my trip feel like a pilgrimage to the top of Mount Everest , which I had never seen . I was able to get a better view of the entire valley . It is a beautiful place to spend a night in and I was able to <unk> the view of the entire valley with the help of the camera , which <unk> and <unk> the entire view with my <unk
